In [1]:
import os
import torch
import numpy as np

from models.solver import Trainer
from data.dataloader import dataloader_info
from utils.io_utils import load_yaml_config, instantiate_from_config
from utils.metric_utils import visualization

/home/harim/Desktop/pyproject/Project_Timeseries_Generation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
configs = load_yaml_config("configs/stock.yaml")
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataloader
dl_info = dataloader_info(configs)

In [4]:
# model
model = instantiate_from_config(configs['model']).to(device)

In [5]:
# train
trainer = Trainer(config=configs, model=model, dataloader=dl_info["dataloader"])
trainer.train()

loss: 0.165091: 100%|██████████| 500/500 [00:12<00:00, 40.76it/s]

training complete


In [6]:
# sampling
dataset = dl_info['dataset']
seq_length, feature_dim = dataset.window, dataset.feature_dim
ori_data = np.load(os.path.join(dataset.dir, f"stock_ground_truth_{seq_length}_train.npy"))
fake_data = trainer.sample(num=len(dataset), size_every=2001, shape=[seq_length, feature_dim])
# if dataset.auto_norm:
#     fake_data = unnormalize_to_zero_to_one(fake_data)
np.save(os.path.join(dataset.dir, f'ddpm_fake_stock.npy'), fake_data)


sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 6, 3], expected input[2001, 24, 6] to have 6 channels, but got 24 channels instead

In [ ]:
# visualization
visualization(ori_data=ori_data, generated_data=fake_data, analysis='pca', compare=ori_data.shape[0])
visualization(ori_data=ori_data, generated_data=fake_data, analysis='tsne', compare=ori_data.shape[0])
visualization(ori_data=ori_data, generated_data=fake_data, analysis='kernel', compare=ori_data.shape[0])